In [1]:
import neo4j
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

c:\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python38\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
load_dotenv()

STORAGE_ACCOUNT_NAME = os.getenv('STORAGE_ACCOUNT_NAME')
STORAGE_ACCOUNT_KEY = os.getenv('STORAGE_ACCOUNT_KEY')

builder = SparkSession.builder\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,\
io.delta:delta-core_2.12:2.4.0,\
io.delta:delta-storage:2.4.0")\
        .config(f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", f"{STORAGE_ACCOUNT_KEY}")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = builder.getOrCreate()

## Product Graphs

In [24]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/neo4j/products/**')\
.withColumnRenamed('2nd_category', 'second_category')\
.withColumnRenamed('3rd_category', 'third_category')\
.withColumnRenamed('4th_category', 'fourth_category')\

df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------+
|      asin|               title|               brand|      main_category|     second_category|      third_category|     fourth_category|  rank|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------+
|B0002WSK8A|Lauren LA100C 39-...|              Lauren|Musical Instruments|             Guitars|Classical &amp; N...|                null|124391|
|B0002FOHTI|Jones JR101MS Obo...|               Jones|Musical Instruments|Instrument Access...|Wind & Woodwind A...|               Reeds|  2949|
|B0002F6K4S|S Series Splash C...|                    |Musical Instruments|  Drums & Percussion|Drum Sets & Set C...|             Cymbals|524988|
|B00061UJBI|Pro Co 3' IPBQ2Q-...|              Pro Co|Musical Instruments|Live Sound &amp; ...|Stage &amp; Studi...|   Instrument 

In [55]:
df.toPandas().to_csv('temp/products.csv', header=True)

In [38]:
from neo4j import GraphDatabase
import json
driver = GraphDatabase.driver('neo4j://localhost:7687', auth=('neo4j', 'test123456'))


In [60]:
driver.execute_query("""
    LOAD CSV WITH HEADERS FROM 'file:///products.csv' as row
    WITH row WHERE row.asin IS NOT NULL
    MERGE (p:Products {asin: row.asin, 
        title: row.title, 
        brand: COALESCE(row.brand, 'not found'),
        main_category: COALESCE(row.main_category, 'not found'),
        second_category: COALESCE(row.second_category, 'not found'), 
        third_category: COALESCE(row.third_category, 'not found'), 
        fourth_category: COALESCE(row.fourth_category, 'not found'),
        rank: COALESCE(row.rank, 'not found')})
    """
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001A08BBFD3D0>, keys=[])